In [1]:
import datasets

/lustrefs/users/shibo.hao/miniforge3/envs/Reasoning360/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the dataset
import os
os.chdir("../..")
dataset_dapo = datasets.load_from_disk("SDSB/deduplicated_dapo_dataset_globally_deduplicated")
dataset_or1 = datasets.load_from_disk("SDSB/or1_dataset_globally_deduplicated")


In [3]:
# find the common column
common_columns = list(set(dataset_dapo.column_names) & set(dataset_or1.column_names))


In [4]:
common_columns

['ability',
 'prompt',
 'reward_model',
 'is_unique',
 'data_source',
 'extra_info',
 'original_question']

In [5]:
# remove "extra_info"
common_columns.remove("extra_info")


In [6]:
columns_to_remove_dapo = [col for col in dataset_dapo.column_names if col not in common_columns]
columns_to_remove_or1 = [col for col in dataset_or1.column_names if col not in common_columns]

dataset_dapo = dataset_dapo.remove_columns(columns_to_remove_dapo)
dataset_or1 = dataset_or1.remove_columns(columns_to_remove_or1)

In [7]:

# merge the dataset
dataset_merged = datasets.concatenate_datasets([dataset_dapo, dataset_or1])

In [8]:
# remove the duplicates with `.is_unique` column
print(dataset_merged)
dataset_merged = dataset_merged.filter(lambda x: x["is_unique"])
print(dataset_merged)

Dataset({
    features: ['data_source', 'prompt', 'ability', 'reward_model', 'original_question', 'is_unique'],
    num_rows: 122972
})


Filter: 100%|██████████| 122972/122972 [00:02<00:00, 47484.35 examples/s]

Dataset({
    features: ['data_source', 'prompt', 'ability', 'reward_model', 'original_question', 'is_unique'],
    num_rows: 117192
})


In [24]:
# get a new dataset, with column "question" and "answer"
# question should be the previous "original_question"
# answer should be the previous x["reward_model"]["ground_truth"]

new_dataset = []
for i in range(len(dataset_merged)):
    new_dataset.append({
        "problem": dataset_merged[i]["original_question"],
        "answer": dataset_merged[i]["reward_model"]["ground_truth"]
    })


In [25]:
len(new_dataset)

117192

In [26]:
final_dataset = []
for d in new_dataset:
    if d['answer'].startswith("[\"") and d['answer'].endswith("\"]"):
        d['answer'] = d['answer'][2:-2]
    if len(d['answer']) < 100:
        final_dataset.append(d)

len(final_dataset)


116632

In [29]:
new_hf_dataset = datasets.Dataset.from_list(new_dataset)

In [30]:
# upload
new_hf_dataset.push_to_hub("SDSB/merged_deduped_dapo_or1_dataset")

Uploading the dataset shards: 100%|██████████| 1/1 [00:05<00:00,  5.04s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/SDSB/merged_deduped_dapo_or1_dataset/commit/5b1768f6bdfd34fbd2c6421faa29259faed400bb', commit_message='Upload dataset', commit_description='', oid='5b1768f6bdfd34fbd2c6421faa29259faed400bb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SDSB/merged_deduped_dapo_or1_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SDSB/merged_deduped_dapo_or1_dataset'), pr_revision=None, pr_num=None)